In [1]:
import pandas as pd
import numpy as np
import plotly.express as ps
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import files
files_uploaded = files.upload()

Saving English_2023_2024_balanced_3000.csv to English_2023_2024_balanced_3000.csv
Saving English_2023_2024_natural_3000 (1).csv to English_2023_2024_natural_3000 (1).csv
Saving English_2023_2024_natural_3000.csv to English_2023_2024_natural_3000.csv


In [3]:
from google.colab import files
files_uploaded = files.upload()

Saving Korea_ESG_Balanced_3000.csv to Korea_ESG_Balanced_3000.csv
Saving Korea_ESG_Natural_3000.csv to Korea_ESG_Natural_3000.csv


In [8]:
df_uk_balanced = pd.read_csv('English_2023_2024_balanced_3000.csv')
df_uk_natural = pd.read_csv('English_2023_2024_natural_3000.csv')
df_kr_balanced = pd.read_csv('Korea_ESG_Balanced_3000.csv')
df_kr_natural = pd.read_csv('Korea_ESG_Natural_3000.csv')

In [5]:
def clean_korea_data(df, version_name):
    """한국 데이터 클리닝"""
    print(f"\n{'='*60}")
    print(f"🇰🇷 CLEANING KOREA {version_name.upper()}")
    print(f"{'='*60}")

    print(f"Initial rows: {len(df)}")

    # 중복 제거
    df_clean = df.drop_duplicates(subset=['title'], keep='first')
    print(f"After removing duplicates: {len(df_clean)}")

    # Null 제거
    df_clean = df_clean.dropna(subset=['title', 'content', 'date'])
    print(f"After removing nulls: {len(df_clean)}")

    # 빈 텍스트 제거
    df_clean = df_clean[df_clean['content'].str.strip() != '']
    df_clean = df_clean[df_clean['analysis_text'].str.strip() != '']
    print(f"After removing empty text: {len(df_clean)}")

    # 컬럼 통일
    df_clean = df_clean.copy()
    df_clean['country'] = 'Korea'
    df_clean = df_clean.rename(columns={'dominant_category': 'category'})
    df_clean['category'] = df_clean['category'].str.strip().str.upper()

    # 날짜 처리
    df_clean['date'] = pd.to_datetime(df_clean['date'])
    df_clean['year'] = df_clean['date'].dt.year
    df_clean['month'] = df_clean['date'].dt.month
    df_clean['year_month'] = df_clean['date'].dt.to_period('M')
    df_clean['quarter'] = df_clean['date'].dt.quarter

    print(f"✅ Complete: {len(df_clean)} articles")
    print(f"   Date range: {df_clean['date'].min().date()} to {df_clean['date'].max().date()}")

    return df_clean

In [6]:
def clean_uk_data(df, version_name):
    """영국 데이터 클리닝 (컬럼명 통일)"""
    print(f"\n{'='*60}")
    print(f"🇬🇧 CLEANING UK {version_name.upper()}")
    print(f"{'='*60}")

    print(f"Initial rows: {len(df)}")

    # Null row 제거 (영국 데이터 특징)
    df_clean = df.dropna(subset=['date', 'title', 'body'])
    print(f"After removing null rows: {len(df_clean)}")

    # 중복 제거
    df_clean = df_clean.drop_duplicates(subset=['title'], keep='first')
    print(f"After removing duplicates: {len(df_clean)}")

    # 빈 텍스트 제거
    df_clean = df_clean[df_clean['body'].str.strip() != '']
    if 'analysis_text' in df_clean.columns:
        df_clean = df_clean[df_clean['analysis_text'].fillna('').str.strip() != '']
    print(f"After removing empty text: {len(df_clean)}")

    # 🔑 컬럼명 통일 (한국 형식에 맞추기!)
    df_clean = df_clean.copy()
    df_clean = df_clean.rename(columns={
        'body': 'content',              # ← 통일!
        'section': 'outlet',            # ← 통일!
        'body_length': 'content_length', # ← 통일!
        'dominant_category': 'category'  # ← 통일!
    })

    # 국가 추가
    df_clean['country'] = 'UK'

    # 카테고리 대문자
    df_clean['category'] = df_clean['category'].str.strip().str.upper()

    # 날짜 처리 (timezone 제거)
    df_clean['date'] = pd.to_datetime(df_clean['date']).dt.tz_localize(None)
    df_clean['year'] = df_clean['date'].dt.year
    df_clean['month'] = df_clean['date'].dt.month
    df_clean['year_month'] = df_clean['date'].dt.to_period('M')
    df_clean['quarter'] = df_clean['date'].dt.quarter

    print(f"✅ Complete: {len(df_clean)} articles")
    print(f"   Date range: {df_clean['date'].min().date()} to {df_clean['date'].max().date()}")

    return df_clean

In [12]:
# 한국 Balanced
kr_balanced_clean = clean_korea_data(df_kr_balanced, "Balanced")

# 한국 Natural
kr_natural_clean = clean_korea_data(df_kr_natural, "Natural")

# 영국 Balanced
uk_balanced_clean = clean_uk_data(df_uk_balanced, "Balanced")

# 영국 Natural
uk_natural_clean = clean_uk_data(df_uk_natural, "Natural")

# 이미 변수 있으면 바로 저장
kr_natural_clean.to_csv('korea_natural_clean.csv', index=False)
uk_natural_clean.to_csv('uk_natural_clean.csv', index=False)
kr_balanced_clean.to_csv('korea_balanced_clean.csv', index=False)
uk_balanced_clean.to_csv('uk_balanced_clean.csv', index=False)

print("✅ All files saved!")

# 파일 하나씩 다운로드
files.download('korea_natural_clean.csv')
files.download('uk_natural_clean.csv')
files.download('korea_balanced_clean.csv')
files.download('uk_balanced_clean.csv')

print("✅ Download complete!")


🇰🇷 CLEANING KOREA BALANCED
Initial rows: 3000
After removing duplicates: 2988
After removing nulls: 2987
After removing empty text: 2987
✅ Complete: 2987 articles
   Date range: 2023-01-01 to 2024-12-31

🇰🇷 CLEANING KOREA NATURAL
Initial rows: 3000
After removing duplicates: 2986
After removing nulls: 2985
After removing empty text: 2985
✅ Complete: 2985 articles
   Date range: 2023-01-01 to 2024-12-31

🇬🇧 CLEANING UK BALANCED
Initial rows: 3000
After removing null rows: 2782
After removing duplicates: 2772
After removing empty text: 2772
✅ Complete: 2772 articles
   Date range: 2023-01-01 to 2024-12-30

🇬🇧 CLEANING UK NATURAL
Initial rows: 3000
After removing null rows: 2781
After removing duplicates: 2776
After removing empty text: 2776
✅ Complete: 2776 articles
   Date range: 2023-01-01 to 2024-12-31
✅ All files saved!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Download complete!


In [ ]:
summary_df = pd.DataFrame({
    'Dataset': ['Korea Balanced', 'Korea Natural', 'UK Balanced', 'UK Natural'],
    'Articles': [
        len(kr_balanced_clean),
        len(kr_natural_clean),
        len(uk_balanced_clean),
        len(uk_natural_clean)
    ],
    'Date Range': [
        f"{kr_balanced_clean['date'].min().date()} ~ {kr_balanced_clean['date'].max().date()}",
        f"{kr_natural_clean['date'].min().date()} ~ {kr_natural_clean['date'].max().date()}",
        f"{uk_balanced_clean['date'].min().date()} ~ {uk_balanced_clean['date'].max().date()}",
        f"{uk_natural_clean['date'].min().date()} ~ {uk_natural_clean['date'].max().date()}"
    ]
})

print(summary_df.to_string(index=False))

       Dataset  Articles              Date Range
Korea Balanced      2987 2023-01-01 ~ 2024-12-31
 Korea Natural      2985 2023-01-01 ~ 2024-12-31
   UK Balanced      2772 2023-01-01 ~ 2024-12-30
    UK Natural      2776 2023-01-01 ~ 2024-12-31


In [ ]:
print("\n" + "="*70)
print("📊 CATEGORY DISTRIBUTION (Natural Versions)")
print("="*70)

print("\n🇰🇷 KOREA Natural:")
kr_cat = kr_natural_clean['category'].value_counts()
print(kr_cat)
print("\nPercentages:")
print(kr_natural_clean['category'].value_counts(normalize=True) * 100)

print("\n🇬🇧 UK Natural:")
uk_cat = uk_natural_clean['category'].value_counts()
print(uk_cat)
print("\nPercentages:")
print(uk_natural_clean['category'].value_counts(normalize=True) * 100)


📊 CATEGORY DISTRIBUTION (Natural Versions)

🇰🇷 KOREA Natural:
category
E    1353
G    1077
S     555
Name: count, dtype: int64

Percentages:
category
E    45.326633
G    36.080402
S    18.592965
Name: proportion, dtype: float64

🇬🇧 UK Natural:
category
E    2240
S     362
G     174
Name: count, dtype: int64

Percentages:
category
E    80.691643
S    13.040346
G     6.268012
Name: proportion, dtype: float64


In [ ]:
# 데이터 준비
comparison_data = pd.DataFrame({
    'Country': ['Korea']*3 + ['UK']*3,
    'Category': ['E', 'S', 'G']*2,
    'Count': [
        (kr_natural_clean['category']=='E').sum(),
        (kr_natural_clean['category']=='S').sum(),
        (kr_natural_clean['category']=='G').sum(),
        (uk_natural_clean['category']=='E').sum(),
        (uk_natural_clean['category']=='S').sum(),
        (uk_natural_clean['category']=='G').sum(),
    ],
    'Percentage': [
        (kr_natural_clean['category']=='E').sum()/len(kr_natural_clean)*100,
        (kr_natural_clean['category']=='S').sum()/len(kr_natural_clean)*100,
        (kr_natural_clean['category']=='G').sum()/len(kr_natural_clean)*100,
        (uk_natural_clean['category']=='E').sum()/len(uk_natural_clean)*100,
        (uk_natural_clean['category']=='S').sum()/len(uk_natural_clean)*100,
        (uk_natural_clean['category']=='G').sum()/len(uk_natural_clean)*100,
    ]
})

# 시각화 1: 개수 비교
fig1 = px.bar(comparison_data, x='Country', y='Count', color='Category',
              title='🌍 ESG Article Count: Korea vs UK (Natural Distribution)',
              barmode='group', height=500,
              color_discrete_map={'E':'#2ecc71', 'S':'#3498db', 'G':'#e74c3c'})
fig1.show()

# 시각화 2: 비율 비교
fig2 = px.bar(comparison_data, x='Country', y='Percentage', color='Category',
              title='🌍 ESG Category Percentage: Korea vs UK',
              barmode='group', height=500,
              color_discrete_map={'E':'#2ecc71', 'S':'#3498db', 'G':'#e74c3c'})
fig2.update_layout(yaxis_title='Percentage (%)')
fig2.show()

In [ ]:
print("\n" + "="*70)
print("📅 MONTHLY PATTERN COMPARISON")
print("="*70)

# 한국 월별
kr_monthly = kr_natural_clean.groupby(['month', 'category']).size().unstack(fill_value=0)
print("\n🇰🇷 Korea - Monthly Distribution:")
print(kr_monthly)

# 영국 월별
uk_monthly = uk_natural_clean.groupby(['month', 'category']).size().unstack(fill_value=0)
print("\n🇬🇧 UK - Monthly Distribution:")
print(uk_monthly)

# Governance만 비교 (한국 3월 급증 vs 영국?)
kr_g_monthly = kr_natural_clean[kr_natural_clean['category']=='G'].groupby('month').size()
uk_g_monthly = uk_natural_clean[uk_natural_clean['category']=='G'].groupby('month').size()

fig3 = go.Figure()
fig3.add_trace(go.Scatter(
    x=kr_g_monthly.index,
    y=kr_g_monthly.values,
    mode='lines+markers',
    name='Korea 🇰🇷',
    line=dict(width=3, color='#e74c3c'),
    marker=dict(size=8)
))
fig3.add_trace(go.Scatter(
    x=uk_g_monthly.index,
    y=uk_g_monthly.values,
    mode='lines+markers',
    name='UK 🇬🇧',
    line=dict(width=3, color='#3498db'),
    marker=dict(size=8)
))
fig3.add_vline(x=3, line_dash="dash", line_color="red",
               annotation_text="한국 주총 시즌", annotation_position="top")
fig3.update_layout(
    title='Governance (G) Articles by Month: Korea vs UK',
    xaxis_title='Month',
    yaxis_title='Number of Articles',
    xaxis=dict(tickmode='linear', dtick=1),
    height=500
)
fig3.show()


📅 MONTHLY PATTERN COMPARISON

🇰🇷 Korea - Monthly Distribution:
category    E    G   S
month                 
1         132  127  50
2          87   81  50
3         135  178  42
4          89   78  44
5         125   70  45
6         119   85  51
7         113   70  54
8         106   61  35
9         105   69  51
10        109   63  31
11        134   84  60
12         99  111  42

🇬🇧 UK - Monthly Distribution:
category    E   G   S
month                
1         203  14  35
2         176  14  27
3         212  15  24
4         170  12  26
5         202  22  32
6         214  16  40
7         245  17  38
8         226  11  35
9         204  15  42
10        147  17  26
11        126  11  24
12        115  10  13


In [ ]:
import pandas as pd
from collections import Counter
import re
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import re

def extract_keywords_v2(titles, language='english', top_n=20):
    """
    개선된 키워드 추출 (불용어 제거)
    """
    # CountVectorizer 사용 (불용어 자동 제거)
    if language == 'english':
        vectorizer = CountVectorizer(
            max_features=top_n,
            stop_words='english',  # 불용어 제거!
            ngram_range=(1, 1),     # 단어 하나씩
            min_df=2,                # 최소 2번 이상 등장
            token_pattern=r'\b[a-zA-Z]{3,}\b'  # 3글자 이상 영문만
        )
    else:  # 한국어
        vectorizer = CountVectorizer(
            max_features=top_n,
            ngram_range=(1, 1),
            min_df=2,
            token_pattern=r'\b\w{2,}\b'
        )

    try:
        X = vectorizer.fit_transform(titles.astype(str))
        words = vectorizer.get_feature_names_out()
        counts = X.sum(axis=0).A1

        word_freq = list(zip(words, counts))
        word_freq.sort(key=lambda x: x[1], reverse=True)

        return word_freq
    except:
        return []

In [ ]:
def extract_ngrams(titles, n=2, top_k=15):
    """N-gram 추출"""
    vectorizer = CountVectorizer(
        ngram_range=(n, n),
        stop_words='english',
        max_features=top_k,
        min_df=2,
        token_pattern=r'\b[a-zA-Z]{3,}\b'
    )

    try:
        X = vectorizer.fit_transform(titles.astype(str))
        ngrams = vectorizer.get_feature_names_out()
        counts = X.sum(axis=0).A1

        ngram_freq = list(zip(ngrams, counts))
        ngram_freq.sort(key=lambda x: x[1], reverse=True)

        return ngram_freq
    except:
        return []

print("\n🇬🇧 UK E - Top Bigrams:")
uk_e_bigrams = extract_ngrams(uk_e['title'], n=2, top_k=15)
for i, (phrase, count) in enumerate(uk_e_bigrams, 1):
    print(f"  {i:2d}. {phrase:35s} ({int(count):3d}회)")

print("\n☀️ UK Summer E - Top Bigrams:")
uk_summer_bigrams = extract_ngrams(uk_summer_e['title'], n=2, top_k=15)
for i, (phrase, count) in enumerate(uk_summer_bigrams, 1):
    print(f"  {i:2d}. {phrase:35s} ({int(count):3d}회)")

print("\n🇬🇧 UK G - Top Bigrams:")
uk_g_bigrams = extract_ngrams(uk_g['title'], n=2, top_k=15)
for i, (phrase, count) in enumerate(uk_g_bigrams, 1):
    print(f"  {i:2d}. {phrase:35s} ({int(count):3d}회)")


🇬🇧 UK E - Top Bigrams:
   1. guardian view                       ( 29회)
   2. climate crisis                      ( 24회)
   3. rishi sunak                         ( 20회)
   4. country diary                       ( 15회)
   5. far right                           ( 13회)
   6. study finds                         ( 13회)
   7. net zero                            ( 11회)
   8. world cup                           ( 11회)
   9. paris olympics                      ( 10회)
  10. report finds                        ( 10회)
  11. ukraine war                         ( 10회)
  12. elon musk                           (  9회)
  13. monday briefing                     (  9회)
  14. premier league                      (  9회)
  15. russia ukraine                      (  9회)

☀️ UK Summer E - Top Bigrams:
   1. paris olympics                      (  6회)
   2. climate crisis                      (  5회)
   3. guardian view                       (  5회)
   4. deep sea                            (  4회)
   5. extreme 

In [ ]:
def count_keyword(titles, keyword):
    """특정 키워드 등장 횟수"""
    return titles.str.lower().str.contains(keyword.lower(), na=False).sum()

# UK E 주요 키워드 체크
print("\n🇬🇧 UK E - Key Environmental Terms:")
keywords_to_check = [
    'climate', 'carbon', 'emission', 'renewable', 'green',
    'fossil', 'net zero', 'warming', 'pollution', 'energy'
]

for kw in keywords_to_check:
    count = count_keyword(uk_e['title'], kw)
    print(f"  {kw:15s}: {count:3d} articles ({count/len(uk_e)*100:4.1f}%)")

# UK Summer E 올림픽 관련
print("\n☀️ UK Summer E - Olympic/Paris Terms:")
summer_keywords = ['olympic', 'paris', 'games', 'sport']
for kw in summer_keywords:
    count = count_keyword(uk_summer_e['title'], kw)
    print(f"  {kw:15s}: {count:3d} articles ({count/len(uk_summer_e)*100:4.1f}%)")

# UK G 의심스러운 키워드
print("\n🤔 UK G - Environmental Keywords (Should be low!):")
env_keywords = ['climate', 'green', 'environment', 'carbon']
for kw in env_keywords:
    count = count_keyword(uk_g['title'], kw)
    if count > 0:
        print(f"  {kw:15s}: {count:3d} articles ({count/len(uk_g)*100:4.1f}%) ⚠️")


🇬🇧 UK E - Key Environmental Terms:
  climate        :  88 articles ( 3.9%)
  carbon         :  24 articles ( 1.1%)
  emission       :  16 articles ( 0.7%)
  renewable      :   6 articles ( 0.3%)
  green          :  68 articles ( 3.0%)
  fossil         :  13 articles ( 0.6%)
  net zero       :  11 articles ( 0.5%)
  warming        :   4 articles ( 0.2%)
  pollution      :  16 articles ( 0.7%)
  energy         :  46 articles ( 2.1%)

☀️ UK Summer E - Olympic/Paris Terms:
  olympic        :  41 articles ( 8.7%)
  paris          :  21 articles ( 4.5%)
  games          :   4 articles ( 0.8%)
  sport          :   3 articles ( 0.6%)

🤔 UK G - Environmental Keywords (Should be low!):
  climate        :   5 articles ( 2.9%) ⚠️
  green          :   4 articles ( 2.3%) ⚠️
  environment    :   1 articles ( 0.6%) ⚠️
  carbon         :   3 articles ( 1.7%) ⚠️


In [ ]:
# 환경 키워드 포함 G 기사 찾기
env_keywords = ['climate', 'green', 'environment', 'carbon', 'emission', 'renewable']

uk_g_with_env = uk_g[
    uk_g['title'].str.lower().str.contains('|'.join(env_keywords), na=False)
]

print(f"\n📊 Found {len(uk_g_with_env)} articles ({len(uk_g_with_env)/len(uk_g)*100:.1f}% of UK G)")
print("\n📰 All titles:")
for i, title in enumerate(uk_g_with_env['title'].values, 1):
    print(f"\n{i:2d}. {title}")

# 더 자세히 보기
print("\n" + "="*70)
print("🔬 DETAILED ANALYSIS")
print("="*70)

for keyword in ['climate', 'green', 'carbon']:
    subset = uk_g[uk_g['title'].str.lower().str.contains(keyword, na=False)]
    if len(subset) > 0:
        print(f"\n🔑 Keyword: '{keyword}' ({len(subset)} articles)")
        for title in subset['title'].values:
            print(f"   - {title}")

# 비교: UK E의 환경 키워드 사용
print("\n" + "="*70)
print("📊 COMPARISON: Environmental Keywords in E vs G")
print("="*70)

comparison = []
for kw in ['climate', 'green', 'carbon', 'environment']:
    e_count = count_keyword(uk_e['title'], kw)
    g_count = count_keyword(uk_g['title'], kw)

    comparison.append({
        'Keyword': kw,
        'E count': e_count,
        'E %': f"{e_count/len(uk_e)*100:.1f}%",
        'G count': g_count,
        'G %': f"{g_count/len(uk_g)*100:.1f}%",
        'Ratio': f"{e_count/max(g_count,1):.1f}x"
    })

import pandas as pd
comparison_df = pd.DataFrame(comparison)
print(comparison_df.to_string(index=False))

print("\n💡 Interpretation:")
print("If E/G ratio is very high (>20x), G articles are likely mislabeled")
print("If ratio is moderate (5-10x), it's Environmental Governance")


📊 Found 13 articles (7.5% of UK G)

📰 All titles:

 1.  UK couple on holiday in Egypt died of carbon monoxide poisoning, coroner rules

 2. Jail term for climate protester, 77, is disproportionate, says Carla Denyer

 3. Oregon county sues major gas provider for allegedly sowing climate doubt

 4. ‘Oh my God, Obama’s stopping by for pasta’: can New York’s Carbone survive its own success?

 5. Key takeaways from the Bonn climate conference

 6. National Grid taps shareholders to help fund £60bn low-carbon energy switch

 7. Uber and Lyft in New York required to be zero-emission by 2030, officials say

 8. Groundbreaking youth-led climate trial comes to an end in Montana

 9. Claire Coutinho: Sunak loyalist will walk a fine green line as energy secretary

10. The Observer view on Cop28: UK is turning its back on chance to lead climate fight | Observer editorial

11. Green power: young environmentalists look to shake up Panama’s politics

12. Roman numerals, wastes of space, and Taylor’s

In [ ]:
# E 카테고리만 필터
kr_e = kr_natural_clean[kr_natural_clean['category'] == 'E']
uk_e = uk_natural_clean[uk_natural_clean['category'] == 'E']

print(f"\n📊 Sample Size:")
print(f"Korea E: {len(kr_e)} articles")
print(f"UK E: {len(uk_e)} articles")

# --- 제목에서 키워드 추출 (간단 버전) ---

def extract_keywords(titles, top_n=20):
    """제목에서 자주 나오는 단어 추출"""
    # 모든 제목 합치기
    all_text = ' '.join(titles.astype(str))
    # 단어 분리 (영어는 공백, 한국어는 조금 다르지만 일단)
    words = re.findall(r'\w+', all_text.lower())
    # 너무 짧은 단어 제거
    words = [w for w in words if len(w) > 2]
    # 빈도 계산
    return Counter(words).most_common(top_n)

print("\n🇰🇷 KOREA - Top 20 Keywords in E Articles:")
kr_e_keywords = extract_keywords(kr_e['title'])
for i, (word, count) in enumerate(kr_e_keywords, 1):
    print(f"  {i:2d}. {word:15s} ({count:3d}회)")

print("\n🇬🇧 UK - Top 20 Keywords in E Articles:")
uk_e_keywords = extract_keywords(uk_e['title'])
for i, (word, count) in enumerate(uk_e_keywords, 1):
    print(f"  {i:2d}. {word:15s} ({count:3d}회)")

# --- 샘플 제목 확인 ---

print("\n📰 Sample Titles (Korea E):")
print(kr_e['title'].sample(5).values)

print("\n📰 Sample Titles (UK E):")
print(uk_e['title'].sample(5).values)


📊 Sample Size:
Korea E: 1353 articles
UK E: 2240 articles

🇰🇷 KOREA - Top 20 Keywords in E Articles:
   1. esg             (185회)
   2. 친환경             (100회)
   3. 글로벌             ( 40회)
   4. 플라스틱            ( 25회)
   5. mou             ( 23회)
   6. 탄소중립            ( 23회)
   7. 에너지             ( 20회)
   8. 솔루션             ( 19회)
   9. 재활용             ( 18회)
  10. 업무협약            ( 18회)
  11. 스타트업            ( 18회)
  12. 중소기업            ( 15회)
  13. a등급             ( 15회)
  14. esg경영           ( 14회)
  15. 태양광             ( 14회)
  16. 2023            ( 14회)
  17. fn마켓워치          ( 14회)
  18. 디지털             ( 13회)
  19. 폐기물             ( 13회)
  20. 플랫폼             ( 13회)

🇬🇧 UK - Top 20 Keywords in E Articles:
   1. the             (884회)
   2. and             (466회)
   3. for             (323회)
   4. with            (124회)
   5. how             (118회)
   6. after           (118회)
   7. new             (114회)
   8. review          (112회)
   9. are             (103회)
  10. from       

In [ ]:
# 7-8월 E 기사
uk_summer_e = uk_natural_clean[
    (uk_natural_clean['month'].isin([7, 8])) &
    (uk_natural_clean['category'] == 'E')
]

# 다른 달 E 기사 (비교용)
uk_other_e = uk_natural_clean[
    (~uk_natural_clean['month'].isin([7, 8])) &
    (uk_natural_clean['category'] == 'E')
]

print(f"\n📊 Article Count:")
print(f"Summer (Jul-Aug) E: {len(uk_summer_e)}")
print(f"Other months E: {len(uk_other_e)}")
print(f"Summer accounts for: {len(uk_summer_e)/len(uk_e)*100:.1f}% of total E articles")

print("\n🔥 Top Keywords in Summer E:")
uk_summer_keywords = extract_keywords(uk_summer_e['title'], top_n=15)
for i, (word, count) in enumerate(uk_summer_keywords, 1):
    print(f"  {i:2d}. {word:15s} ({count:3d}회)")

print("\n📰 Sample Summer E Titles:")
print(uk_summer_e['title'].sample(min(5, len(uk_summer_e))).values)

# 날짜별 분포 (여름 내에서도)
print("\n📅 Daily pattern in July-August:")
uk_summer_e_daily = uk_summer_e.groupby('date').size()
print(f"Peak day: {uk_summer_e_daily.idxmax()} with {uk_summer_e_daily.max()} articles")
print(f"Average per day: {uk_summer_e_daily.mean():.1f} articles")


📊 Article Count:
Summer (Jul-Aug) E: 471
Other months E: 1769
Summer accounts for: 21.0% of total E articles

🔥 Top Keywords in Summer E:
   1. the             (182회)
   2. and             ( 99회)
   3. for             ( 66회)
   4. after           ( 32회)
   5. review          ( 30회)
   6. olympic         ( 28회)
   7. with            ( 25회)
   8. are             ( 23회)
   9. new             ( 23회)
  10. says            ( 21회)
  11. how             ( 20회)
  12. paris           ( 20회)
  13. who             ( 20회)
  14. out             ( 19회)
  15. climate         ( 19회)

📰 Sample Summer E Titles:
['India to revise colonial-era penal code and toughen laws protecting women'
 'Baklâ review – this teasing tomfoolery is devilishly smart'
 '‘We are facing dependence on China’: EU battles to support green industry'
 'Top pilots’ union sounds alarm as regulators consider smaller crew sizes'
 'No more ‘Jasper disaster’: Philipsen is now sprint king in a fast-moving business | William Fotheringham'

In [ ]:
# G 카테고리만 필터
kr_g = kr_natural_clean[kr_natural_clean['category'] == 'G']
uk_g = uk_natural_clean[uk_natural_clean['category'] == 'G']

print(f"\n📊 Sample Size:")
print(f"Korea G: {len(kr_g)} articles ({len(kr_g)/len(kr_natural_clean)*100:.1f}%)")
print(f"UK G: {len(uk_g)} articles ({len(uk_g)/len(uk_natural_clean)*100:.1f}%)")

print("\n🇰🇷 KOREA - Top 20 Keywords in G Articles:")
kr_g_keywords = extract_keywords(kr_g['title'])
for i, (word, count) in enumerate(kr_g_keywords, 1):
    print(f"  {i:2d}. {word:15s} ({count:3d}회)")

print("\n🇬🇧 UK - Top 20 Keywords in G Articles:")
uk_g_keywords = extract_keywords(uk_g['title'])
for i, (word, count) in enumerate(uk_g_keywords, 1):
    print(f"  {i:2d}. {word:15s} ({count:3d}회)")

print("\n📰 Sample Titles (Korea G):")
print(kr_g['title'].sample(min(10, len(kr_g))).values)

print("\n📰 Sample Titles (UK G):")
print(uk_g['title'].sample(min(10, len(uk_g))).values)

# 한국 3월 G vs 다른 달 G
kr_march_g = kr_g[kr_g['month'] == 3]
kr_other_g = kr_g[kr_g['month'] != 3]

print(f"\n🎯 Korea March G Focus:")
print(f"March G: {len(kr_march_g)} articles")
print(f"Other months G: {len(kr_other_g)} articles")

print("\n📰 Korea March G Sample Titles (주총 관련?):")
print(kr_march_g['title'].sample(min(10, len(kr_march_g))).values)


📊 Sample Size:
Korea G: 1077 articles (36.1%)
UK G: 174 articles (6.3%)

🇰🇷 KOREA - Top 20 Keywords in G Articles:
   1. esg             (145회)
   2. 글로벌             ( 29회)
   3. 업무협약            ( 25회)
   4. esg경영           ( 24회)
   5. iso             ( 19회)
   6. 지배구조            ( 18회)
   7. 사외이사            ( 17회)
   8. 카카오             ( 16회)
   9. 경쟁력             ( 15회)
  10. 중소기업            ( 14회)
  11. 밸류업             ( 13회)
  12. 이사회             ( 13회)
  13. 이복현             ( 13회)
  14. 국민연금            ( 13회)
  15. 리스크             ( 13회)
  16. 신년사             ( 11회)
  17. 활성화             ( 11회)
  18. 중진공             ( 11회)
  19. 영업익             ( 11회)
  20. 신사업             ( 11회)

🇬🇧 UK - Top 20 Keywords in G Articles:
   1. the             ( 74회)
   2. and             ( 38회)
   3. for             ( 26회)
   4. with            ( 14회)
   5. review          ( 13회)
   6. how             ( 13회)
   7. says            ( 12회)
   8. you             ( 12회)
   9. but             ( 11회)
  1

In [ ]:
# E 카테고리 비교
print("\n🌱 ENVIRONMENT (E):")
print(f"  Korea: {len(kr_e)} articles (45.3%)")
print(f"  UK: {len(uk_e)} articles (80.7%)")
print(f"  → UK is 2.1x more focused on Environment")

# G 카테고리 비교
print("\n🏛️ GOVERNANCE (G):")
print(f"  Korea: {len(kr_g)} articles (36.1%)")
print(f"  UK: {len(uk_g)} articles (6.3%)")
print(f"  → Korea is 6.2x more focused on Governance")

# 계절성
print("\n📅 SEASONALITY:")
print(f"  Korea March G: {len(kr_march_g)} articles (16.5% of all Korea articles)")
print(f"  UK has no significant G spike in any month")

print("\n✅ Analysis complete!")
print("="*70)


🌱 ENVIRONMENT (E):
  Korea: 1353 articles (45.3%)
  UK: 2240 articles (80.7%)
  → UK is 2.1x more focused on Environment

🏛️ GOVERNANCE (G):
  Korea: 1077 articles (36.1%)
  UK: 174 articles (6.3%)
  → Korea is 6.2x more focused on Governance

📅 SEASONALITY:
  Korea March G: 178 articles (16.5% of all Korea articles)
  UK has no significant G spike in any month

✅ Analysis complete!
